In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import wandb
import math

import folium
from geopy.geocoders import Nominatim
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [ ]:
df = pd.DataFrame()  

import random
random.seed(0)
for file in random.sample(filenames, 20):
    if df.empty:
        df = pd.read_csv(os.path.join(dirname, file))
    else:
        d = pd.read_csv(os.path.join(dirname, file))
        df = pd.concat([df, d])
df

In [ ]:
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 
files = glob.glob(path + "/*.csv")

csv_list = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0)
    district_id = filename.split("/")[4].split(".")[0]
    df["district_id"] = district_id
    csv_list.append(df)
    
engagement_data = pd.concat(csv_list)
engagement_data = engagement_data.reset_index(drop=True)
engagement_data.head()

In [ ]:
engagement_data = engagement_data.rename(columns = {'lp_id':'LP ID'})

In [ ]:
district_data = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
district_data.head()

In [ ]:
district_data = district_data[district_data.isnull().sum(axis=1)<5]
district_data

In [ ]:
district_data.isnull().sum()

In [ ]:
district_data.dtypes

In [ ]:
product_data = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
product_data.head()

In [ ]:
product_data.isnull().sum()/len(product_data.index)

In [ ]:
df_merge = pd.merge(engagement_data, product_data, how='inner', on='LP ID')
df_merge['district_id'] = df_merge['district_id'].astype('int')
df_master = pd.merge(df_merge, district_data, how='inner', on='district_id')
df_master.head()

In [ ]:
round(df_master.isnull().sum()/len(df_master.index),2)

In [ ]:
df_master.dropna(axis=0,inplace=True)

In [ ]:
df_master_clean = df_master[:]

In [ ]:
print('before:' ,df_master_clean['pct_black/hispanic'].unique())
def clean_x(x):
    x1= x.split(',')[1][:4].strip()
    if x1 == '1[':
        x1 = x1[:1]
    return float(x1)
    
df_master_clean['pct_black/hispanic'] = df_master_clean['pct_black/hispanic'].apply(clean_x)
print('after:' ,df_master_clean['pct_black/hispanic'].unique())

print('before:' ,df_master_clean['pct_free/reduced'].unique())
df_master_clean['pct_free/reduced'] = df_master_clean['pct_free/reduced'].apply(clean_x)
print('after:' ,df_master_clean['pct_free/reduced'].unique())

In [ ]:
df_master_clean.drop(['county_connections_ratio','URL'],axis=1,inplace=True)

In [ ]:
print('before:' ,df_master_clean['pp_total_raw'].unique())
df_master_clean['pp_total_raw'] = df_master_clean['pp_total_raw'].apply(lambda x: int(x[1:-1].split(',')[1].strip()))
print('after:' ,df_master_clean['pp_total_raw'].unique())

In [ ]:
df_master_clean['pp_total_raw'] = df_master_clean['pp_total_raw'].astype('int')

In [ ]:
df_master_clean.head()

In [ ]:
round(df_master_clean.isnull().sum()/len(df_master_clean.index),2)

In [ ]:
df_master_clean['time'] = pd.to_datetime(df_master_clean['time'], format='%Y-%m-%d')
df_master_clean['month'] = df_master_clean['time'].apply(lambda x: x.month)
df_master_clean['month'] = df_master_clean['month'].astype('category')
df_master_clean['month'] = df_master_clean['month'].cat.rename_categories(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])

In [ ]:
df_master_clean.loc[:, ['LP ID', 'Product Name','district_id','Provider/Company Name']].value_counts()

In [ ]:
state_df = df_master_clean['state'].value_counts()
state_df = state_df.to_frame()
state_df.reset_index(level=0,inplace=True)
plt.figure(figsize=(16,5))
ax = sns.barplot(y='state', x='index', data=state_df)
plt.xlabel('State')
plt.ylabel('Count')
plt.title('State Analysis')
ax.grid(True)


plt.show()
state_df.T

In [ ]:
locale_df = df_master_clean['locale'].value_counts()
locale_df = locale_df.to_frame()
locale_df.reset_index(level=0,inplace=True)
plt.figure(figsize=(10,8))
ax = sns.barplot(y='locale', x='index', data=locale_df)
plt.xlabel('Locale')
plt.ylabel('Count')
plt.title('Locale Analysis')
ax.grid(True)
plt.show()
locale_df

In [ ]:
d1 = df_master_clean.pivot_table(values='engagement_index',
                                            index='state',
                                            columns='locale',
                                            aggfunc='sum')

ax = d1.plot.bar(figsize=(20, 6),logy=True).grid(True)
plt.ylabel('Count')
plt.xlabel('State and  Locale')
plt.title("Engagement anlysis based on State with locale")
plt.show()
d1.T

In [ ]:
d0 = df_master_clean.pivot_table(values='engagement_index',
                                            index='state',
                                            aggfunc='sum')

ax = d0.plot.bar(figsize=(20, 6),logy=True, color='g').grid(True)
plt.xlabel('State')
plt.ylabel('Count')
plt.title('State Analysis vs engagement Index')
plt.show()
d0.T

In [ ]:
m1 = df_master_clean.pivot_table(values='engagement_index',
                                            index='month',
                                            aggfunc='sum')
ax = m1.plot.bar(figsize=(20, 6),logy=True, color='r').grid(True)
plt.xlabel('Months')
plt.ylabel('Count')
plt.title("Engagement vs Month analysis")
plt.show()
# m1.T

m2 = df_master_clean.pivot_table(values='pct_access',
                                            index='month',
                                            aggfunc='sum')
ax = m2.plot.bar(figsize=(20, 6),logy=True, color='g').grid(True)
plt.xlabel('Months')
plt.ylabel('Count')
plt.title("Percentage of students accessed product vs Month analysis")
plt.show()

In [ ]:
d2 = df_master_clean.pivot_table(values='engagement_index',
                                            index='Product Name',
                                            aggfunc='sum')
d2_sorted = d2.sort_values(by='engagement_index',ascending=False).head(20)

ax = d2_sorted.plot.bar(figsize=(20, 6),logy=True).grid(True)
plt.xlabel('Products')
plt.ylabel('Count')
plt.title("Product vs Engagement analysis")
plt.show()
d2_sorted.T

In [ ]:
comp_name_df = df_master_clean['Provider/Company Name'].value_counts()
comp_name_df = comp_name_df.to_frame()
comp_name_df.reset_index(level=0,inplace=True)
comp_name_df = comp_name_df.head(20)

plt.figure(figsize=(10,12))
ax = sns.barplot(x='Provider/Company Name', y='index', data=comp_name_df)
plt.xlabel('Count')
plt.ylabel('Companies')
plt.title('Top Companies vs Engangement analysis')
ax.grid(True)
plt.show()

In [ ]:
sector_df = df_master_clean['Sector(s)'].value_counts()
sector_df = sector_df.to_frame()
sector_df.reset_index(level=0,inplace=True)
plt.figure(figsize=(12,5))
ax = sns.barplot(y='Sector(s)', x='index', data=sector_df)
plt.ylabel('Count')
plt.xlabel('Sectors')
plt.title('Sectors Analysis')
ax.grid(True)
plt.show()
sector_df.T

In [ ]:
df_master_clean['Primary Essential Function categories'] = df_master_clean['Primary Essential Function'].apply(lambda x: x.split('-')[0])
df_master_clean['Primary Essential Function categories'].value_counts()

In [ ]:
type_df = df_master_clean['pct_black/hispanic'].value_counts()
type_df = type_df.to_frame()
type_df.reset_index(level=0,inplace=True)
# pef_df1 = pef_df.head(10)
plt.figure(figsize=(10,6))
ax = sns.barplot(y='pct_black/hispanic', x='index', data=type_df)
plt.ylabel('Count')
plt.xlabel('pct_black/hispanic')
plt.title('pct_black/hispanic Analysis')
plt.xticks(rotation=90) 
ax.grid(True)
plt.show()

In [ ]:
type_df1 = df_master_clean.pivot_table(values='engagement_index',
                                            index='pct_black/hispanic',
                                            aggfunc='sum')

type_df1.reset_index(level=0,inplace=True)

plt.figure(figsize=(10,6))
ax = sns.barplot(y='engagement_index', x='pct_black/hispanic', data=type_df1)
plt.ylabel('Count')
plt.xlabel('pct_black/hispanic')
plt.title('pct_black/hispanic Analysis')
plt.xticks(rotation=90) 
ax.grid(True)
plt.show()